<a href="https://colab.research.google.com/github/Blessedtomato/TotoPrediction/blob/main/Toto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#Import the relevant libraries and packages

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Xisiqomelir/xisiqo_datadump/main/ToTo.csv")
#Load the required dataset using pandas’s read CSV function

In [3]:
df.head()
#Taking a closer look at the data took help of “head()”function

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number,From Last,...,Division 3 Winners,Division 3 Prize,Division 4 Winners,Division 4 Prize,Division 5 Winners,Division 5 Prize,Division 6 Winners,Division 6 Prize,Division 7 Winners,Division 7 Prize
0,3884,2023-07-06,13,16,28,34,38,40,47,"34,40",...,147.0,1666.0,263.0,508.0,6591.0,50.0,7890.0,25.0,121282.0,10.0
1,3883,2023-07-03,14,30,34,40,45,49,12,30,...,85.0,2223.0,235.0,439.0,4664.0,50.0,6423.0,25.0,87982.0,10.0
2,3882,2023-06-29,5,15,16,30,37,46,49,5,...,515.0,1661.0,1208.0,387.0,25404.0,50.0,33315.0,25.0,475093.0,10.0
3,3881,2023-06-26,4,5,17,28,39,44,31,28,...,237.0,1741.0,531.0,424.0,12396.0,50.0,16290.0,25.0,232715.0,10.0
4,3880,2023-06-22,3,9,28,32,37,46,7,"28,32,46",...,112.0,2163.0,358.0,369.0,6596.0,50.0,11867.0,25.0,125106.0,10.0


In [4]:
df.info()
#Check attributes names and datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Draw                1544 non-null   int64  
 1   Date                1544 non-null   object 
 2   Winning Number 1    1544 non-null   int64  
 3   2                   1544 non-null   int64  
 4   3                   1544 non-null   int64  
 5   4                   1544 non-null   int64  
 6   5                   1544 non-null   int64  
 7   6                   1544 non-null   int64  
 8   Additional Number   1544 non-null   int64  
 9   From Last           915 non-null    object 
 10  Same As Day         208 non-null    float64
 11  Odd                 1544 non-null   int64  
 12  Even                1544 non-null   int64  
 13   1-10               1544 non-null   int64  
 14   11-20              1544 non-null   int64  
 15   21-30              1544 non-null   int64  
 16   31-40

In [5]:
df.Date = pd.to_datetime(df.Date)
#Converting Date column to datetime type

**DATA PROCESSING**

In [6]:
df.drop(['From Last', 'Same As Day', 'Odd', 'Even', ' 1-10', ' 11-20', ' 21-30', ' 31-40', ' 41-50', 'Division 1 Winners', 'Division 1 Prize', 'Division 2 Winners', 'Division 2 Prize', 'Division 3 Winners', 'Division 3 Prize', 'Division 4 Winners', 'Division 4 Prize', 'Division 5 Winners', 'Division 5 Prize', 'Division 6 Winners', 'Division 6 Prize', 'Division 7 Winners', 'Division 7 Prize'], axis=1, inplace=True)
df.head()
#Dropping unwanted features

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number
0,3884,2023-07-06,13,16,28,34,38,40,47
1,3883,2023-07-03,14,30,34,40,45,49,12
2,3882,2023-06-29,5,15,16,30,37,46,49
3,3881,2023-06-26,4,5,17,28,39,44,31
4,3880,2023-06-22,3,9,28,32,37,46,7


In [7]:
df.drop(['Date', 'Draw'], axis=1, inplace=True)
df.head()

,Winning Number 1,2,3,4,5,6,Additional Number
0,13,16,28,34,38,40,47
1,14,30,34,40,45,49,12
2,5,15,16,30,37,46,49
3,4,5,17,28,39,44,31
4,3,9,28,32,37,46,7


In [8]:
df = df[:-654]
#To remove draws before 9/10/2014 because they are 45 numbers before

In [9]:
df.tail()

,Winning Number 1,2,3,4,5,6,Additional Number
885,5,22,25,28,33,34,32
886,8,11,17,21,41,49,35
887,4,9,31,34,40,44,35
888,19,28,31,43,45,48,41
889,11,12,25,28,34,47,15


In [10]:
print(df.shape)
#890 records, 7 fields

(890, 7)


In [11]:
df.describe()

,Winning Number 1,2,3,4,5,6,Additional Number
count,890.000000,890.000000,890.000000,890.000000,890.000000,890.000000,890.000000
mean,6.823596,14.055056,21.270787,28.601124,35.632584,43.100000,25.334831
std,5.564764,7.350941,7.960090,8.061867,7.301236,5.420257,14.228974
min,1.000000,2.000000,4.000000,7.000000,14.000000,19.000000,1.000000
25%,3.000000,9.000000,15.000000,23.000000,31.000000,40.000000,13.000000
50%,5.000000,13.000000,21.000000,29.000000,37.000000,45.000000,26.000000
75%,9.000000,19.000000,27.000000,35.000000,41.000000,47.000000,37.000000
max,33.000000,38.000000,43.000000,47.000000,48.000000,49.000000,49.000000


Deep learning algorithms expect all input features to vary in a similar way, and ideally to have a mean of 0, and a variance of 1. We must re-scale our data so that it fulfills these requirements.

In [12]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [13]:
transformed_df.head()
#Scaled data

,0,1,2,3,4,5,6
0,1.110537,0.264733,0.845844,0.670057,0.324431,-0.572250,1.523465
1,1.290340,2.170322,1.600028,1.414720,1.283712,1.089121,-0.937687
2,-0.327888,0.128620,-0.662524,0.173615,0.187391,0.535331,1.664103
3,-0.507691,-1.232515,-0.536827,-0.074606,0.461471,0.166137,0.398367
4,-0.687495,-0.688061,0.845844,0.421836,0.187391,0.535331,-1.289281


In [14]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

890

In [85]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length

7

In [86]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

7

Crate X and y for each row in the scaled data. It should have format for Keras LSTM model (rows, window size, balls)

In [87]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [88]:
X.shape

(883, 7, 7)

In [89]:
y.shape

(883, 7)

In [90]:
X[0]

array([[ 1.11053721,  0.2647331 ,  0.84584431,  0.6700572 ,  0.32443094,
        -0.57225014,  1.52346549],
       [ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123, -0.9376874 ],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078,  1.6641028 ],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714,  0.39836703],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078, -1.28928067],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078,  1.59378414],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123, -0.37513817]])

In [92]:
y[0]

array([-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
       -1.31063742, -1.28928067])

In [93]:
X[1]

array([[ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123, -0.9376874 ],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078,  1.6641028 ],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714,  0.39836703],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078, -1.28928067],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078,  1.59378414],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123, -0.37513817],
       [-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
        -1.31063742, -1.28928067]])

In [94]:
y[1]

array([-0.1480851 ,  0.94530056,  1.22293637,  0.42183622, -0.08668942,
        0.90452441, -0.44545682])

In [95]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.15, random_state = 31)

In [96]:
model = Sequential()
#Initialise RNN

In [97]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add the input layer and the LSTM layer

In [98]:
model.add(Dropout(0.2))
#Add a first Dropout layer in order to reduce overfitting

In [99]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add a second LSTM layer

In [100]:
model.add(Dropout(0.2))
#Add a second Dropout layer

In [101]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add a third LSTM layer

In [102]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))
#Add a fourth LSTM layer

In [103]:
model.add(Dense(59))
#Dense layer

In [104]:
model.add(Dense(number_of_features))
#Add the last output layer

In [115]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.00001), loss ='mse', metrics=['accuracy'])
#Compiling the RNN

In [ ]:
history=model.fit(X, y, batch_size=10, epochs=800, verbose=1) #validation_data=(X_test, y_test))
#Low batch size, high epochs, Low learning rate will increase accuracy

Epoch 1/800
89/89 [==============================] - 28s 151ms/step - loss: 0.9872 - accuracy: 0.2061
Epoch 2/800
89/89 [==============================] - 14s 158ms/step - loss: 0.9860 - accuracy: 0.2016
Epoch 3/800
89/89 [==============================] - 14s 153ms/step - loss: 0.9855 - accuracy: 0.1993
Epoch 4/800
89/89 [==============================] - 14s 154ms/step - loss: 0.9854 - accuracy: 0.2016
Epoch 5/800
89/89 [==============================] - 14s 154ms/step - loss: 0.9847 - accuracy: 0.2095
Epoch 6/800
89/89 [==============================] - 14s 153ms/step - loss: 0.9837 - accuracy: 0.2084
Epoch 7/800
89/89 [==============================] - 13s 151ms/step - loss: 0.9836 - accuracy: 0.2106
Epoch 8/800
89/89 [==============================] - 14s 161ms/step - loss: 0.9838 - accuracy: 0.2027
Epoch 9/800
89/89 [==============================] - 14s 153ms/step - loss: 0.9831 - accuracy: 0.2084
Epoch 10/800
89/89 [==============================] - 14s 154ms/step - loss: 0.982

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, None, 480)        476160    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, None, 480)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 480)        1384320   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, None, 480)         0         
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 480)        1384320   
 nal)                                                            
                                                        

In [52]:
to_predict = df.head(15)
to_predict

,Winning Number 1,2,3,4,5,6,Additional Number
0,13,16,28,34,38,40,47
1,14,30,34,40,45,49,12
2,5,15,16,30,37,46,49
3,4,5,17,28,39,44,31
4,3,9,28,32,37,46,7
5,2,25,28,30,32,46,48
6,6,15,17,42,46,49,20
7,2,22,28,33,34,36,7
8,6,21,31,32,35,48,19
9,5,16,26,28,32,34,24


In [53]:
to_predict = np.array(to_predict)
to_predict

array([[13, 16, 28, 34, 38, 40, 47],
       [14, 30, 34, 40, 45, 49, 12],
       [ 5, 15, 16, 30, 37, 46, 49],
       [ 4,  5, 17, 28, 39, 44, 31],
       [ 3,  9, 28, 32, 37, 46,  7],
       [ 2, 25, 28, 30, 32, 46, 48],
       [ 6, 15, 17, 42, 46, 49, 20],
       [ 2, 22, 28, 33, 34, 36,  7],
       [ 6, 21, 31, 32, 35, 48, 19],
       [ 5, 16, 26, 28, 32, 34, 24],
       [ 7, 15, 33, 40, 45, 49,  5],
       [ 5, 13, 30, 34, 36, 48, 20],
       [ 4, 15, 17, 35, 42, 48, 45],
       [ 1,  4,  7,  8, 34, 44, 12],
       [ 7, 13, 18, 19, 25, 29, 36]])

In [54]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

array([[ 1.11053721,  0.2647331 ,  0.84584431,  0.6700572 ,  0.32443094,
        -0.57225014,  1.52346549],
       [ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123, -0.9376874 ],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078,  1.6641028 ],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714,  0.39836703],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078, -1.28928067],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078,  1.59378414],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123, -0.37513817],
       [-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
        -1.31063742, -1.28928067],
       [-0.1480851 ,  0.94530056,  1.22293637,  0.42183622, -0.08668942,
         0.90452441, -0.44545682],
       [-0.32788828,  0.2647

In [55]:
y_pred = model.predict(np.array([scaled_to_predict]))
print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 37ms/step
The predicted numbers in the last lottery game are: [ 7 13 21 29 35 43 26]
